In [5]:
import pandas as pd
import numpy as np
import warnings

import pylab 
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.colors import ListedColormap
%matplotlib inline
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
from graphviz import Source
from IPython.display import Image

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_val_predict, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score

from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from mlxtend.classifier import StackingClassifier

AttributeError: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

In [9]:
redwinequality = "Resources/winequality-red.csv"

In [10]:
df_redwinequality = pd.read_csv(redwinequality, encoding="ISO-8859-1")

In [11]:

df_redwinequality_new = df_redwinequality[['volatile acidity','sulphates','citric acid','alcohol','residual sugar','fixed acidity','chlorides','free sulfur dioxide','pH','quality']].copy()

In [12]:
df_redwinequality_new = df_redwinequality_new.rename(columns = {'volatile acidity': 'volatile_acidity', 'citric acid': 'citric_acid','residual sugar':'residual_sugar', 'fixed acidity':'fixed_acidity','free sulfur dioxide':'free_sulfur_dioxide'}, inplace = False)
df_redwinequality_new.head()

,volatile_acidity,sulphates,citric_acid,alcohol,residual_sugar,fixed_acidity,chlorides,free_sulfur_dioxide,pH,quality
0,0.70,0.56,0.00,9.4,1.9,7.4,0.076,11.0,3.51,5
1,0.88,0.68,0.00,9.8,2.6,7.8,0.098,25.0,3.20,5
2,0.76,0.65,0.04,9.8,2.3,7.8,0.092,15.0,3.26,5
3,0.28,0.58,0.56,9.8,1.9,11.2,0.075,17.0,3.16,6
4,0.70,0.56,0.00,9.4,1.9,7.4,0.076,11.0,3.51,5


In [13]:
df_redwinequality_new['quality_label'] = df_redwinequality_new.quality.apply(lambda q: '1' if q <= 5 else '2' if q < 7 else '3')
df_redwinequality_new.head()

,volatile_acidity,sulphates,citric_acid,alcohol,residual_sugar,fixed_acidity,chlorides,free_sulfur_dioxide,pH,quality,quality_label
0,0.70,0.56,0.00,9.4,1.9,7.4,0.076,11.0,3.51,5,1
1,0.88,0.68,0.00,9.8,2.6,7.8,0.098,25.0,3.20,5,1
2,0.76,0.65,0.04,9.8,2.3,7.8,0.092,15.0,3.26,5,1
3,0.28,0.58,0.56,9.8,1.9,11.2,0.075,17.0,3.16,6,2
4,0.70,0.56,0.00,9.4,1.9,7.4,0.076,11.0,3.51,5,1


TensorFlow

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras import Sequential
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
df_redwinequality_new.isna().sum() if (df.isna().sum()).any()>0 else "Data is Clean"

In [ ]:

df_redwinequality_new['quality'].mean()

In [ ]:

train_dataset = df.sample(frac=0.8,random_state=0)
test_dataset = df.drop(train_dataset.index)

In [ ]:
train_labels = train_dataset.pop('quality')
test_labels = test_dataset.pop('quality')

In [ ]:
normed_train_data = normalize(train_dataset)
normed_test_data = normalize(test_dataset)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(128, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
  ])

  #optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer='adam',
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [ ]:

plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [Quality]')

In [ ]:
# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=1000, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])